# Phát hiện biển số xe sử dụng phương pháp YOLO
Bài toán sử dụng phương pháp YOLO để phát hiện ra vùng chứa biển số xe.

Từ đó tiếp tục làm các bước:
- phân đoạn ký tự trong biển số xe
- nhận dạng ký tự bằng CNN

In [13]:
from lp_detection import lp_detect_using_yolo
from character_segmentation import segment_characters
from lp_recognition import lp_recognize

import pandas as pd
import os
import cv2
from keras.models import load_model


## Test dataset
Gồm 186 ảnh
- labels.xlsx chứa giá trị biển số đúng

In [14]:
# đọc các labels
labels = pd.read_excel('test_dataset/labels.xlsx', engine='openpyxl')
labels['ID'] = labels['ID'].map(str)
# in thử 10 giá trị
labels[0:10]


,ID,NUMBER
0,1,KL55R2473
1,2,GJW115A1138
2,3,KL16J3636
3,4,MH20EE7598
4,5,PJJ1L76
5,6,HR26DA2330
6,7,HR268P3543
7,8,HR26CT6Y02
8,9,KL01BR8055
9,10,KL7BZ99


## Khởi tạo các tham số của YOLO

In [15]:
# Khởi tạo các tham số
confThreshold = 0.5 # ngưỡng tin cậy
nmsThreshold = 0.4  # Ngưỡng ngăn chặn không tối đa

inputWidth = 416 # width của ảnh đầu vào mạng YOLO
inputHeight = 416 # height của ảnh đầu vào mạng YOLO

# tải các tên của classes và nối thành một list
classesFile = str(r"yolo_utils/classes.names")
classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

# cung cấp các tệp cấu hình của mạng YOLO
modelConfiguration = str(r"yolo_utils/darknet-yolov3.cfg")
modelWeights = str(r"yolo_utils/lapi.weights")

# khởi tạo model YOLO
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [16]:
# đọc ảnh
folder_path = str(r"test_dataset/images/")
file_list = os.listdir(folder_path)

# đọc model CNN
loaded_model = load_model('weights/best_model.h5')

count = 0  # biến chỉ số dự đoán chính xác

# duyệt toàn bộ ảnh
for path in file_list:
    # lấy ra ID
    id = path[:-4]
    # lấy ra biển số
    number_plate = labels['NUMBER'].where(labels['ID'] == id).dropna().values[0]
    # đọc ảnh
    image = cv2.imread(folder_path + path)

    # phát hiện vùng chứa biển số
    img_plate = lp_detect_using_yolo(net=net, frame=image, confThreshold=confThreshold, nmsThreshold=nmsThreshold, inputWidth=inputWidth, inputHeight=inputHeight)

    # nếu tìm được biển
    if img_plate is not None:
        # phân đoạn ký tự
        char_list = segment_characters(image=img_plate)

        # nếu không phân tách được ký tự
        if len(char_list) == 0:
            continue

        # nhận dạng ký tự
        predict_plate = lp_recognize(model=loaded_model, char_list=char_list)

        if predict_plate == number_plate:
            count += 1

# độ chính xác trên toàn bộ dataset
print("Accuracy: " + str((count/166)*100) + "%")


Accuracy: 56.62650602409639%
